# Debug Drill: The Backwards Coefficients

**Scenario:**
Your colleague built a linear regression model to predict customer spend. They're confused by the results.

"The model performs well," they say, "but the coefficients don't make sense. It says more orders means LESS spend? That can't be right!"

**Your Task:**
1. Run the notebook
2. Find the bug (hint: why do the coefficients have wrong signs?)
3. Fix it
4. Write a 3-bullet postmortem

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
DATA_URL = 'https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/'

df = pd.read_csv(DATA_URL + 'streamcart_customers.csv')
print(f"Loaded {len(df):,} customers")

In [ ]:
# ===== COLLEAGUE'S CODE (CONTAINS BUG) =====

# Feature engineering
df['spend_per_order'] = df['total_spend'] / (df['orders_total'] + 1)  # Avoid divide by zero

# Select features - colleague wants to predict total_spend
features = [
    'tenure_months',
    'logins_last_30d', 
    'orders_total',
    'avg_order_value',
    'spend_per_order'    # <-- Colleague added this derived feature
]

X = df[features].fillna(0)
y = df['total_spend']

# Split and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Performance:")
print(f"  MAE: ${mae:.2f}")
print(f"  R²: {r2:.3f}")

print(f"\nCoefficients:")
for feature, coef in zip(features, model.coef_):
    print(f"  {feature:20s}: {coef:+.4f}")

print(f"\n🤔 Wait... orders_total has a NEGATIVE coefficient?")
print(f"   That means more orders = less spend? That doesn't make sense!")

---

## Your Investigation

Something is wrong. The coefficients have illogical signs.

### Step 1: What's the problem?

In [ ]:
# TODO: Investigate the features
# Hint: Check the correlation matrix. Are any features suspiciously correlated?

print("=== Correlation Matrix ===")
correlation_matrix = df[features + ['total_spend']].corr()
print(correlation_matrix.round(3))

In [ ]:
# TODO: Check how spend_per_order is calculated
# What happens when you divide total_spend by orders_total?

print("=== How spend_per_order relates to total_spend ===")
print(f"spend_per_order = total_spend / (orders_total + 1)")
print(f"\nThis means: total_spend = spend_per_order × (orders_total + 1)")
print(f"\n🔍 What do you notice about this relationship?")

In [ ]:
# What do you notice? Write your diagnosis:

diagnosis = """
YOUR DIAGNOSIS HERE:

The problem is...

This is called...

Why it causes wrong coefficient signs...

"""
print(diagnosis)

### Step 2: Fix the code

In [ ]:
# TODO: Fix the feature selection - remove the problematic feature

features_fixed = [
    # List only non-problematic features here
]

# Uncomment and complete:
# X_fixed = df[features_fixed].fillna(0)
# X_train_fixed, X_test_fixed, y_train_fixed, y_test_fixed = train_test_split(
#     X_fixed, y, test_size=0.2, random_state=42
# )
# 
# model_fixed = LinearRegression()
# model_fixed.fit(X_train_fixed, y_train_fixed)
# 
# y_pred_fixed = model_fixed.predict(X_test_fixed)
# mae_fixed = mean_absolute_error(y_test_fixed, y_pred_fixed)
# r2_fixed = r2_score(y_test_fixed, y_pred_fixed)
# 
# print(f"Fixed Model Performance:")
# print(f"  MAE: ${mae_fixed:.2f}")
# print(f"  R²: {r2_fixed:.3f}")
# 
# print(f"\nFixed Coefficients:")
# for feature, coef in zip(features_fixed, model_fixed.coef_):
#     print(f"  {feature:20s}: {coef:+.4f}")

In [ ]:
# ============================================
# SELF-CHECK: Did you fix it?
# ============================================

# Uncomment after fixing:
# 
# # The orders_total coefficient should now be POSITIVE
# orders_idx = features_fixed.index('orders_total')
# assert model_fixed.coef_[orders_idx] > 0, "orders_total coefficient should be positive!"
# 
# # The tenure coefficient should also be positive
# tenure_idx = features_fixed.index('tenure_months')
# assert model_fixed.coef_[tenure_idx] > 0, "tenure_months coefficient should be positive!"
# 
# print("✓ Coefficients now have sensible signs!")
# print("✓ More orders = more spend (makes sense!)")
# print("✓ Longer tenure = more spend (makes sense!)")

### Step 3: Verify the fix with visualization

In [ ]:
# TODO: After fixing, uncomment to visualize coefficient comparison

# fig, axes = plt.subplots(1, 2, figsize=(14, 5))
# 
# # Buggy model coefficients
# colors_buggy = ['green' if c > 0 else 'red' for c in model.coef_]
# axes[0].barh(features, model.coef_, color=colors_buggy, alpha=0.7)
# axes[0].axvline(x=0, color='black', linewidth=0.5)
# axes[0].set_xlabel('Coefficient')
# axes[0].set_title('BUGGY Model: Wrong Signs!')
# 
# # Fixed model coefficients  
# colors_fixed = ['green' if c > 0 else 'red' for c in model_fixed.coef_]
# axes[1].barh(features_fixed, model_fixed.coef_, color=colors_fixed, alpha=0.7)
# axes[1].axvline(x=0, color='black', linewidth=0.5)
# axes[1].set_xlabel('Coefficient')
# axes[1].set_title('FIXED Model: Sensible Signs!')
# 
# plt.tight_layout()
# plt.show()

### Step 4: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: Backwards Coefficients Bug

### What happened:
- (Your answer: Describe the symptom you observed)

### Root cause:
- (Your answer: Why did this happen? What's the technical name for this problem?)

### How to prevent:
- (Your answer: What checks would catch this in the future?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lesson:** 

1. **Multicollinearity** happens when features are highly correlated with each other.

2. **Derived features that contain the target** (like `spend_per_order = total_spend / orders`) create a mathematical relationship that breaks coefficient interpretation.

3. **Always check:** "Is this feature derived from my target variable?"

4. **Red flag:** Coefficients with signs that contradict domain knowledge ("more orders = less spend") almost always indicate multicollinearity or data leakage.

---

## Bonus Challenge

Can you find another problematic feature combination? What if you included both `avg_order_value` and `orders_total` when predicting `total_spend`?

In [ ]:
# Bonus: Investigate the relationship between avg_order_value, orders_total, and total_spend

print("Think about it:")
print("  total_spend ≈ avg_order_value × orders_total")
print("\nIf you use BOTH as features to predict total_spend, what might happen?")